In [ ]:
%load_ext autoreload
%autoreload 2

<a name="top"></a>
# <center>CCR/Ghub Jupyter Notebook Demo</center>

## Abstract

- This Jupyter Notebook provides a demonstration for executing jobs in parallel on the University at Buffalo (UB)'s Center For Computational Research (CCR)'s generally accessible high performance compute cluster, UB-HPC, on CCR via sbatch and Slurm commands and on Ghub via the Pegasus Workflow Management System (WMS).

- This tool's GitHub repository is located at https://github.com/GhubGateway/CCR_Ghub_Demo.

### CCR Specifics

- Login to CCR OnDemand.
- Launch the Quick Launch JupyterLab/Notebook Interactive App, check the JupyterLab check box, and Connect to Jupyter.
- Open the ccrghubdemo.ipynb notebook.
- The ccrghubdemo_kernel Kernel is set by default.
- Select Kernel, Restart Kernel and Run All Cells. 

### Ghub Specifics

- Login to Ghub.
- Launch the Jupyter Notebooks (202210) tool.
- Open the ccrghubdemo.ipynb notebook.
- Set the Python 3 (ipykernel) Kernel.
- Select Kernel, Restart & Run All.

## Overview

- Select the ISMIP6 Mapped Collection Folder and Modeling Groups. Click the `Run Workflow` button to run the workflow jobs which summerizes and displays time series information from experiment files contained within the selected modeling groups. 


In [ ]:
# Get the name of the operating system and set the HOST.
with open("/etc/os-release") as f:
    d = {}
    for line in f:
        k,v = line.rstrip().split("=")
        d[k] = v
id = d.get("ID")
print ('ID: %s' %str(id))
if id == 'flatcar':
    HOST='CCR'
elif id == 'debian':
    HOST = 'Ghub'
else:
    HOST = 'Unknown'
    print ('Unable to detect the Host.')
    print ('/ect/os-release: ', d)
    print ('Notebook Run Workflow button is disabled.')
print ('Detected Host: %s' %HOST)

In [ ]:
# Setup and preoprocessing:

import sys
import os
import getpass
import platform
import shutil
import atexit
import math
import time
import subprocess

import numpy as np
import pandas as pd
import hublib
#print (help(hublib))
import hublib.ui as ui
#print (help(ui))
import functools
import ctypes as ct 


np.set_printoptions(threshold=np.inf) 

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output

#print(sys.path)

# Set up the environment for this notebook

# Setup paths to executables
scriptpath = os.path.realpath(" ")
        
# Directory where this notebook resides
self_tooldir = os.path.dirname(scriptpath)

# Setup path to python and bash scripts
self_bindir = os.path.join(self_tooldir, "bin")

# Add to PYTHONPATH
sys.path.insert (1, self_bindir)

# Setup path to python and bash scripts
self_datadir = os.path.join(self_tooldir, "data")

# Setup path to python and bash scripts
self_remotebindir = os.path.join(self_tooldir, "remotebin")

self_username = getpass.getuser()

if HOST == 'CCR':
    
    from run_jobs_ccr import RunJobs
    
elif HOST == 'Ghub':
    
    import hublib.use
    #print (help(hublib.use))
    
    # Version of Pegasus
    %use pegasus-5.0.1
    
    from run_jobs_ghub import RunJobs

# Configuration parameters

import Configuration as cfg
if cfg.VERBOSE == True:
    print ('cfg.DISPERSION_MODEL: ', cfg.DISPERSION_MODEL, '\n')

self_log_filepath = os.path.join(self_tooldir, 'ccrghubdemo_log_file.txt')
self_log_backup_filepath = os.path.join(self_tooldir, 'ccrghubdemo_log_backup_file.txt')

widget_border_style = '1px solid black'
widget_output_border_style = '1px solid black'

BOLD = '\033[1m'
SUCCESS = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
END = '\033[0m'

BORDER_STYLE = '1px solid black'

BUTTON_WIDTH = '400px'
BUTTON_HEIGHT = '30px'

DROPDOWN_WIDTH = '965px'
DROPDOWN_HEIGHT = '30px'
button2_width = '150px'
button2_height = '30px'

# Clean up: remove files from the data/results folder and the bin/__pycache__ folder
def exit_handler():
    
    for file in os.listdir(self_tooldir):
        
        if os.path.isfile(file):
            if file.endswith('.txt'):
                if file != 'README.txt' and file.endswith('netcdf_info.txt') == False and file != 'ccrghubdemo_log_file.txt':
                    #print ("Deleting: %s\n" %file)
                    os.remove(file)
            if file.endswith(".yml"):
                #print ("Deleting: %s\n" %file)
                os.remove(file)
            elif file.endswith(".stdout"):
                #print ("Deleting: %s\n" %file)
                os.remove(file)
            elif file.endswith(".stderr"):
                #print ("Deleting: %s\n" %file)
                os.remove(file)

    #dirpath = os.path.join(self_bindir, "__pycache__")
    #if (os.path.exists(dirpath)):
        #print ("Deleting: %s\n" %dirpath)
        #shutil.rmtree(dirpath)
        
    FH1.flush()
    FH1.close()

atexit.register(exit_handler);   

In [ ]:
#help ('modules')

In [ ]:
# prevent In[] and Out[] from displaying on left
#HTML('''
#<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>
#''')

In [ ]:
#https://api.jquery.com/ready/
HTML('''
<script>
    function scroll_to_top() {
        Jupyter.notebook.scroll_to_top();
    } 
    $( window ).on( "load", scroll_to_top() );
</script>
''')

In [ ]:
# Button styles
HTML('''
<style>.buttontextclass { color:black ; font-size:130%}</style>
''')

In [ ]:
#https://stackoverflow.com/questions/36757301/disable-ipython-notebook-autoscrolling

In [ ]:
# returning Javascript Error: Can't find variable: IPython
#%%javascript
#IPython.OutputArea.prototype._should_scroll = function(lines) {
    #return false;
#}


In [ ]:
# Initialize

if os.path.exists(self_log_filepath):
    shutil.move (self_log_filepath, self_log_backup_filepath)

FH1 = open(self_log_filepath, 'w')

show_log_output_button = widgets.Button(description="Show Log Output", disabled=False,\
    layout=widgets.Layout(width=BUTTON_WIDTH, height=BUTTON_HEIGHT),\
    style= {'button_color':'skyblue','font_weight':'bold'}).add_class("buttontextclass")


# Utility Functions

def log_info (message):
    
    if show_log_output_button.description == 'Hide Log Output': 
        with log_output:
            print (message)    
    FH1.write('%s\n' %message)
    FH1.flush()

def log_status (output_widget, message):
    
    with output_widget:
        print (message)
    log_info (message)
    
def log_success (output_widget, message):
    
    with output_widget:
        print ('%s%s%s' %(SUCCESS,message,END))
    log_info (message)
    
def log_warning (output_widget, message):
    
    with output_widget:
        print ('%s%s%s' %(WARNING,message,END))
    log_info (message)
    
def log_error (output_widget, message):
    
    with output_widget:
        print ('%s%s%s' %(FAIL,message,END))
    log_info (message)
    
if (1): #cfg.VERBOSE == True:
    
    log_info ('Operating System Platform: ' + platform.system() + ' ' + platform.release())
    log_info ('\n')

    log_info ('Environment:\n')
    log_info ('scriptpath: ' + scriptpath)
    log_info ('tooldir: ' + self_tooldir)
    log_info ('bindir: ' + self_bindir)
    log_info ('username: ' + self_username)
    log_info ('\n')
    
    #print (type(sys.path)) # <class 'list'>
    #print (sys.path)
    log_info ('sys.path: ' + ' '.join(str(path)+'\n' for path in sys.path))
    log_info ('\n')
    
    #print (type(os.environ["PATH"])) # <class 'str'>
    #print (os.environ["PATH"])
    log_info ('os.environ["PATH"]: ' + os.environ["PATH"])
    log_info ('\n')
    
log_info ('Python version: %s' %str(platform.python_version()))
#help ('modules')

In [ ]:
# Setup the show file buttons

def show_file_header(template):
    
    display(HTML('<h3>Contents of file %s:</h3>' %template))
    
def show_download_button(download_button_output=None, filepath=None):
    
    with download_button_output:
        
        #print ('show_download_button filepath: ', filepath)
        # Download file from Ghub
        display(HTML('<h3>Download %s to Your Web Browser</h3>' %os.path.basename(filepath)))
        #print (os.path.relpath(self_log_filepath, self_tooldir))
        downloadTXTButton = ui.Download(os.path.relpath(filepath, self_tooldir),
            label = 'Download File', style='success', icon='fa-arrow-circle-down')
        display(downloadTXTButton)
    
def show_file_output(button, descr=None, filepath=None, file_output=None, download_button_output=None):

    log_info ('show_file_output...')
        
    log_info ('button: %s' %str(button))
    log_info ('descr: %s' %descr)
    log_info ('filepath: %s' %filepath)
    
    if os.path.exists(filepath):

        if button.description == 'Show %s' %descr:

            button.description = 'Hide %s' %descr

            with file_output:
                
                show_file_header(filepath)

                f = open(filepath,'r')
                for line in f:
                    print(line.rstrip())
                f.close()

                if HOST == 'Ghub' and 'Invoke Script' not in descr:
                    show_download_button(download_button_output, filepath)
        else:

            button.description = 'Show %s' %descr
            file_output.clear_output()
            download_button_output.clear_output()
    else:
        log_error (file_output, '%s does not exist ' %filepath + '. Please contact us.')


In [ ]:
def show_file_output_button(descr, filepath, disabled):
    button = widgets.Button(description='Show %s' %descr, disabled=disabled,\
        layout=widgets.Layout(width=BUTTON_WIDTH, height=BUTTON_HEIGHT),\
        style= {'button_color':'skyblue','font_weight':'bold'}).add_class('buttontextclass')
    file_output = widgets.Output(layout={'border': BORDER_STYLE})
    download_button_output = widgets.Output()
    button.on_click\
        (functools.partial(show_file_output, descr=descr, filepath=filepath, file_output=file_output, download_button_output=download_button_output))
    display (button)
    display (file_output)
    display (download_button_output)
    return button, file_output, download_button_output


<a name="userguide"></a>
## User Guide

### [**Steps for using this tool**](#steps_for_using_this_tool)<br />

1. [Select the Mapped Collection Folder](#step_1) <br />
2. [Select the Modeling Groups](#step_2)<br />
3. [Run the Workflow](#step_3)<br />
4. [View Workflow Progress](#step_4)<br />
5. [View Workflow Results](#step_5)<br />
6. [View Log Output](#step_6)<br />
7. [View Scripts and Source Code](#step_7)<br />

Please [Schedule a Meeting](https://theghub.org/bookmeeting) if you have questions.

In [ ]:
# Get the UB CCR's map collection folders information 
mapped_collections_filename = os.path.join(self_datadir, 'ub-ccr-ghub-ISMIP6-mapped_collections.xlsx')

mapped_collections_df = pd.read_excel (mapped_collections_filename)
#print (type(mapped_collections_df))
#print (mapped_collections_df)

num_mapped_collections = len(mapped_collections_df)
#print (num_mapped_collections)

HBox_layout = widgets.Layout(height='45px', width='98%', display='flex', flex_flow='row', justify_content='flex-start')

def append_modeling_groups (modeling_group):
    model_checkbox = widgets.Checkbox(
        value = True,
        description = modeling_group,
        indent = False,
        disabled = True,
        style = {'description_width':'200px'},
        layout= {'height': '40px', 'width': '200px'})
    model = widgets.HBox(children=[model_checkbox], layout = HBox_layout)
    all_modeling_groups.append(model)

folder_list = []
description_list = []
modeling_groups_list = []

for i in range(num_mapped_collections):
    folder = str(mapped_collections_df['Folder'][i].strip(' \t\n\r'))
    folder_list.append(folder)
    description = str(mapped_collections_df['Description'][i].strip(' \t\n\r'))
    description_list.append(description)
    modeling_groups = (str(mapped_collections_df['Modeling Groups'][i].strip(' \t\n\r')))
    modeling_groups_list.append (modeling_groups)
log_info ('folder_list: ' + str(folder_list))
log_info ('description_list: ' + str (description_list))
log_info ('modeling_groups_list: ' + str (modeling_groups_list))

folder_index = 0
all_modeling_groups = []


<a name="step_1"></a>
## Step 1: Select the Mapped Collection Folder [&#8607;](#userguide)

Select one of six predetermined UB CCR ISMIP6 mapped collection folders.


In [ ]:
def folder_dropdown_callback(change):
    
    global folder_index
   
    if change['type'] == 'change' and change['name'] == 'value' and change['new'] != ' ' \
        and folder_dropdown.value != None:
        
        selected_folder = folder_dropdown.value
        log_info ('selected folder: ' + selected_folder)
        folder_index = folder_list.index(selected_folder)
        initialize()
            
folder_dropdown = widgets.Dropdown(
    description = 'Folder:',
    disabled = False,
    options = folder_list,
    value = folder_list[0],
    style = {'description_width': '150px'},
    layout = widgets.Layout(width=DROPDOWN_WIDTH, height=DROPDOWN_HEIGHT)
)
folder_dropdown.observe(folder_dropdown_callback)


In [ ]:
folder_form = ui.Form([folder_dropdown], name = 'Mapped Collection Folder')
display (folder_form)

<a name="step_2"></a>
## Step 2: Select the Modeling Groups [&#8607;](#userguide)

Select the modeling groups within the selected mapped collection folder. Time series data from experiment files contained within the selected modeling groups are summerized and displayed when the workflow is executed. See the `Run the Workflow` section for more information. By default, the AWI and ITLS_PIK modeling groups are selected.

In [ ]:
def select_default_modeling_groups():
    
    # Reduce the computational load by default
    global all_modeling_groups
    for i in range(len(all_modeling_groups)):
        modeling_group = all_modeling_groups[i].children[0].description
        if modeling_group == 'AWI' or modeling_group == 'ILTS_PIK':
            all_modeling_groups[i].children[0].value = True
        else:
            all_modeling_groups[i].children[0].value = False
        
def select_default_models_button_callback(p):
    select_default_modeling_groups()
    
select_default_models_button = widgets.Button(description="Select Defaults", disabled=False,\
                             layout=widgets.Layout(width=button2_width, height=button2_height, justify_content="flex-end"),\
                             style= {'button_color':'lightgray','font_weight':'bold'})
select_default_models_button.add_class("button2textclass")
select_default_models_button.on_click (select_default_models_button_callback)

def select_all_modeling_groups():
    global all_modeling_groups
    for i in range(len(all_modeling_groups)):
        all_modeling_groups[i].children[0].value = True
        
def select_all_models_button_callback(p):
    select_all_modeling_groups()
    
select_all_models_button = widgets.Button(description="Select All", disabled=False,\
                             layout=widgets.Layout(width=button2_width, height=button2_height, justify_content="flex-end"),\
                             style= {'button_color':'lightgray','font_weight':'bold'})
select_all_models_button.add_class("button2textclass")
select_all_models_button.on_click (select_all_models_button_callback)

def unselect_all_modeling_groups():
    global all_modeling_groups
    for i in range(len(all_modeling_groups)):
        all_modeling_groups[i].children[0].value = False
        
def unselect_all_models_button_callback(p):
    unselect_all_modeling_groups()
    
unselect_all_models_button = widgets.Button(description="Unselect All", disabled=False,\
                             layout=widgets.Layout(width=button2_width, height=button2_height, justify_content="flex-end"),\
                             style= {'button_color':'lightgray','font_weight':'bold'})
unselect_all_models_button.add_class("button2textclass")

unselect_all_models_button.on_click (unselect_all_models_button_callback)

In [ ]:
models_form_output = widgets.Output(layout={'border': widget_border_style})
display(models_form_output)

In [ ]:
models_form_output = widgets.Output(layout={'border': widget_border_style})
display(models_form_output)

<a name="step_3"></a>
## Step 3: Run the Workflow [&#8607;](#userguide)

Click the `Run Workflow` button to run the workflow jobs.  

- The get_netcdf_info.py Python script is executed in parallel for each of the selected modeling groups. This script uses the Python xarray package to analyze time series data from experiment files contained within the mapped collection folder's selected modeling groups and creates a json file for each of the selected modeling groups. The process_netcdf_info.py Python script reads the json files created by get_netcdf_info.py, determines unique time series information for each experiment type, and creates a text file containing the summerized time series information. This text file is displayed in the `View Workflow Results` section when the workflow completes.

- On CCR, execution of the Python scripts are inititated by the run_jobs_ccr.py script in the tool's bin directory. Bash scripts invoke the sbatch command line tool to submit batch jobs to Slurm and to monitor when the submitted jobs complete. See [Running Jobs](https://docs.ccr.buffalo.edu/en/latest/hpc/jobs/) to learn more about using sbatch and Slurm on CCR.

- On Ghub, the Python scripts are encapsulated as a workflow by the run_jobs_ghub.py script in the tool's bin directory. The Pegasus WMS automates and manages the execution of the workflow jobs, including staging the jobs, distributing the work, submitting the jobs to run in parallel on CCR's UB-HPC compute cluster, as well as handling data flow dependencies and overcoming job failures. Launch the Ghub [Pegasus Workflows Tutorial and Templates](https://theghub.org/tools/ghubex1) tool to learn more about running Pegasus WMS workflows on Ghub.

- If an error is encountered while running the workflow, the cause of the error will be written to the log output file, ccrghubdemo_log_file.txt. See the `View Log Output File` section for more information.


In [ ]:
 # Run Workflow

def run_workflow(p):
    
    # print (p) #Button    
    
    global self_workflow_succeeded
    self_workflow_succeeded = False
    global self_workflow_results_filepath
    
    workflow_progress.clear_output()
    workflow_results.clear_output()
        
    with workflow_progress:
        
        disable_widgets()
        
        start_time = time.time()

        try:
            
            log_info ('folder_index: ' + str(folder_index))
            
            selected_modeling_groups = []
            for i in range(len(all_modeling_groups)):
                if all_modeling_groups[i].children[0].value == True:
                      selected_modeling_groups.append (all_modeling_groups[i].children[0].description)
                        
            if len(selected_modeling_groups) > 0:
                
                ice_sheet = folder_dropdown.value.split('/')[-1]
            
                self_workflow_results_filepath = os.path.join(self_tooldir, '%s_processed_netcdf_info.txt' %ice_sheet)
                log_info ('self_workflow_results_filepath: ' + self_workflow_results_filepath)
                
                for file in os.listdir(self_tooldir):
                    if os.path.isfile(file):
                        if file.startswith('ccrghubdemo-') and file.endswith('_out.txt'):
                            os.remove(file)
                        if file.endswith('netcdf_info.json'):
                            os.remove(file)
                        if file.endswith('netcdf_info.txt'):
                            os.remove(file)
 
                #Note: Execution time depends on the current UB CCR workload.
                log_status (workflow_progress, "Pegasus workflow in progress...")
            
                instance = RunJobs (self_username, description_list[folder_index], folder_list[folder_index], ','.join(selected_modeling_groups))
                print ('instance: %s' %str(instance))
                exitCode = instance.run_jobs()
                del instance

                log_status (workflow_progress, "\nWorkflow elapsed time: " + str((time.time() - start_time)/60.0) + " minutes\n")

                # Check if the results files were created and transferred from CCR 
                # to determine if workflow completed successfully

                if os.path.exists(self_workflow_results_filepath):

                    log_status (workflow_progress, "Workflow completed successfully\n")
                    self_workflow_succeeded = True

                    with workflow_results:

                        print("%s: \n\n" %self_workflow_results_filepath)
                        f = open(self_workflow_results_filepath,'r')
                        for line in f:
                            print(line.rstrip())
                        f.close()

                else:

                    log_error (workflow_progress, "Workflow did not complete successfully")
                    log_error (workflow_progress, "%s not generated by the workflow\n" %self_workflow_results_filepath)
                    self_workflow_succeeded = False

                    filepath = os.path.join(self_tooldir, 'pegasus.analysis')
                    if (os.path.exists(filepath)):
                        print("pegasus.analysis:\n")
                        f = open(filepath, 'r')
                        output = f.read()
                        f.close()
                        print (output)
                
                finish_workflow_processing()
                    
            else:
                
                log_error (workflow_progress, '\nERROR: No modeling groups are selected. Please select at least one modeling group.')

        except Exception as e:
        
            log_error (workflow_progress, "Workflow Exception: %s\n" %str(e))
            
        enable_widgets()
            

In [ ]:
runWorkflowButton = widgets.Button(description="Run Workflow", disabled=False,\
    layout=widgets.Layout(width=BUTTON_WIDTH, height=BUTTON_HEIGHT),\
    style= {'button_color':'lawngreen','font_weight':'bold'}).add_class("buttontextclass")
runWorkflowButton.on_click (run_workflow)
#help (runWorkflowButton)
display(runWorkflowButton)


<a name="step_4"></a>
## Step 4: View Workflow Progress [&#8607;](#userguide)


In [ ]:
workflow_progress = widgets.Output(layout={'border': widget_output_border_style})
display(workflow_progress)

<a name="step_5"></a>
## Step 5: View Workflow Results [&#8607;](#userguide)


In [ ]:
workflow_results = widgets.Output(layout={'border': widget_output_border_style})
display(workflow_results)

In [ ]:
def send_user_email(workflow_succeeded):

    email_subject = 'Ghub session #' + session_num + '.'
    
    if workflow_succeeded:
        email_text = 'Your ccrghubdemo jobs are complete!\r'
        email_text = email_text+'\rOutput files can be accessed on theghub.org in the following directory:'
        email_text = email_text+'\r' + str(self_tooldir)
    else:
        email_text = 'Your ccrghubdemo jobs Failed.'
        email_text = email_text+'\rPlease check theghub.org for further information, in the directory:'
        email_text = email_text+'\r' + str(self_tooldir)        
        
    email_cmd = 'submit --progress silent mail2self -t "'+email_text+'" -s "'+email_subject+'"'
    
    # email debugging
    #start_time = time.time()
    os.system(email_cmd)
    #elapsed_time = time.time() - start_time
    #print ('email elapsed time: ', elapsed_time)
    
def finish_workflow_processing():
    
    try:

        log_info ('\nfinish_workflow_processing...')
        
        # workflow.yml is created by Wrapper.py
        #filepath = os.path.join(self_tooldir, 'workflow.yml')
        #if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            #os.remove(filepath)

        for file in os.listdir(self_tooldir):
            if os.path.isfile(file):
                
                if file[0] == ".":
                    if file != ".gitattributes":
                        #print ("Deleting: %s\n" %file)
                        os.remove(file)

                #if file.startswith('python') and file.endswith('.stdout'):
                    #log_info ('\n%s:\n' %file)
                    #f = open(file,'r')
                    #for line in f:
                        #log_info (line)
                    #f.close()
                    #os.remove(file)
                    
                if file.startswith('python') and file.endswith('.stderr'):
                    log_info ('\n%s:\n' %file)
                    f = open(file,'r')
                    for line in f:
                        log_info (line)
                    f.close()
                    os.remove(file)
         
        filepath = os.path.join(self_tooldir, 'pegasus.analysis')
        if (os.path.exists(filepath)):
            filesize = os.path.getsize(filepath)
            log_info ('\npegasus.analysis filesize: ' + str(filesize))
            log_info ('pegasus.analysis:\n')
            f = open(filepath, 'r')
            output = f.read()
            f.close()
            log_info (output)
            os.remove(filepath)
        
        filepath = os.path.join(self_tooldir, "pegasusstatus.txt")
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        filepath = os.path.join(self_tooldir, "pegasusjobstats.csv")
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        filepath = os.path.join(self_tooldir, "pegasussummary-time.csv")
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        filepath = os.path.join(self_tooldir, "pegasussummary.csv")
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        # send email to user
        #if HOST == 'Ghub':
           #send_user_email(self_workflow_succeeded)
                
        log_info ('finish_workflow_processing done.')
        
    except Exception as e:
        log_error (workflow_progress, "EXCEPTION: %s\n" % str(e))


<a name="step_6"></a>
## Step 6: View Log Output File [&#8607;](#userguide)

- If an error is encountered while running this tool,
the cause of the error will be written to the log output file, ccrghubdemo_log_file.txt.

- Click the `Show Log Output File` button to view the log output file.


In [ ]:
def show_log_output(change):
    
    if os.path.exists(self_log_filepath):
            
        if show_log_output_button.description == 'Show Log Output':
        
            show_log_output_button.description = 'Hide Log Output'
        
            with log_output:
            
                print("%s: \n\n" %self_log_filepath)
                f = open(self_log_filepath,'r')
                for line in f:
                    print(line.rstrip())
                f.close()
        else:
        
            show_log_output_button.description = 'Show Log Output'
            log_output.clear_output()
    else:
        log_error (log_output, '%s does not exist ' %self_log_filepath + '. Please contact us.')

show_log_output_button.on_click(show_log_output)
display (show_log_output_button)

In [ ]:
log_output = widgets.Output(layout={'border': '1px solid black'})
display (log_output)

<a name="step_7"></a>
## Step 7: View the Scripts [&#8607;](#userguide)

- Click a button below to view a script. On Ghub, when a button is clicked, a Download button will also display to enable you to download the script or source code to your web browser.


### Python Source Code

In [ ]:
b1_descr = 'get_netcdf_info.py'
b1_filepath = os.path.join(self_tooldir, 'bin', '%s' %b1_descr)
b1_button, b1_file_output, b1_download_button_output = \
    show_file_output_button (b1_descr, b1_filepath, False)

b2_descr = 'process_netcdf_info.py'
b2_filepath = os.path.join(self_tooldir, 'bin', '%s' %b2_descr)
b2_button, b2_file_output, b2_download_button_output = \
    show_file_output_button (b2_descr, b2_filepath, False)

### Python Installation Scripts and Python Launch Script

On CCR, a Python virtual environment and a kernel that uses the virtual environment were installed for this tool. See [Using Python at CCR](https://docs.ccr.buffalo.edu/en/latest/howto/python/) to learn more about installing Python virtual environments and kernels on CCR.


In [ ]:
b3_descr = 'Install Python Environment Script'
b3_filepath = os.path.join(self_tooldir, 'install_ccrghubdemo.2023.01_python_environment.sh')
b3_button, b3_file_output, b3_download_button_output = \
    show_file_output_button (b3_descr, b3_filepath, False)

b4_descr = 'Install Python Kernel Script'
b4_filepath = os.path.join(self_tooldir, 'install_ccrghubdemo.2023.01_kernel.sh')
b4_button, b4_file_output, b4_download_button_output = \
    show_file_output_button (b4_descr, b4_filepath, False)

b5_descr = 'Python Launch Script'
b5_filepath = os.path.join(self_tooldir, 'bin', 'pythonLaunch.sh')
b5_button, b5_file_output, b5_download_button_output = \
    show_file_output_button (b5_descr, b5_filepath, False)

### CCR Scripts



In [ ]:
b6_descr = 'run_jobs_ccr.py'
b6_filepath = os.path.join(self_tooldir, 'bin', '%s' %b6_descr)
b6_button, b6_file_output, b6_download_button_output = \
    show_file_output_button (b6_descr, b6_filepath, False)

b7_descr = 'get_netcdf_info_sbatch_slurm.sh'
b7_filepath = os.path.join(self_tooldir, 'bin', '%s' %b7_descr)
b7_button, b7_file_output, b7_download_button_output = \
    show_file_output_button (b7_descr, b7_filepath, False)

b8_descr = 'get_netcdf_info_slurm.sh'
b8_filepath = os.path.join(self_tooldir, 'bin', '%s' %b8_descr)
b8_button, b8_file_output, b8_download_button_output = \
    show_file_output_button (b8_descr, b8_filepath, False)

b9_descr = 'process_netcdf_info_sbatch_slurm.sh'
b9_filepath = os.path.join(self_tooldir, 'bin', '%s' %b9_descr)
b9_button, b9_file_output, b9_download_button_output = \
    show_file_output_button (b9_descr, b9_filepath, False)

b10_descr = 'process_netcdf_info_slurm.sh'
b10_filepath = os.path.join(self_tooldir, 'bin', '%s' %b10_descr)
b10_button, b10_file_output, b10_download_button_output = \
    show_file_output_button (b10_descr, b10_filepath, False)

b11_descr = 'wait.sh'
b11_filepath = os.path.join(self_tooldir, 'bin', '%s' %b11_descr)
b11_button, b11_file_output, b11_download_button_output = \
    show_file_output_button (b11_descr, b11_filepath, False)



### Ghub Script



In [ ]:
b12_descr = 'run_jobs_ghub.py'
b12_filepath = os.path.join(self_tooldir, 'bin', '%s' %b12_descr)
b12_button, b12_file_output, b12_download_button_output = \
    show_file_output_button (b12_descr, b12_filepath, False)



In [ ]:
### Ghub Python Script

In [ ]:
def disable_widgets():

    global all_modeling_groups

    #modeling_groups_selection_form.disabled = True
    # Children need to be explictly disabled.
    for i in range(len(all_modeling_groups)):
        all_modeling_groups[i].children[0].disabled = True
    runWorkflowButton.disabled = True
    
def enable_widgets():
    
    global all_modeling_groups

    #modeling_groups_selection_form.disabled = False
    # Children need to be explictly enabled.
    for i in range(len(all_modeling_groups)):
        all_modeling_groups[i].children[0].disabled = False
    runWorkflowButton.disabled = False
         
def initialize():
    
    global all_modeling_groups
    
    disable_widgets()
    
    # Display form
    with models_form_output:
        
        modeling_groups = list(modeling_groups_list[folder_index].split(','))
        all_modeling_groups = []
        for i in range (len(modeling_groups)):
            append_modeling_groups(modeling_groups[i])
        modeling_groups_checkbox_form = ui.Form(all_modeling_groups, name = '')
        modeling_groups_buttons_form = ui.Form([select_default_models_button, select_all_models_button, unselect_all_models_button], name = '')
        modeling_groups_selection_form = ui.Form([modeling_groups_checkbox_form, modeling_groups_buttons_form], name = 'Modeling Group Selections')
        #Reduce the computational load by default
        select_default_modeling_groups()
        
        clear_output()
        display(modeling_groups_selection_form)
    
    if HOST != 'Unknown':
        enable_widgets()


In [ ]:
# Start processing.
initialize()
